<a href="https://colab.research.google.com/github/RogerHeederer/FineTuning/blob/main/BERT_NaverSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference https://github.com/kimwoonggon/publicservant_AI

##전체 시나리오
1. 네이버 감성 분석 데이터 Load & Preprocess
2. BERT 인풋 만들기
3. 버트를 활용한 감성분석 모델 만들기
4. 훈련 / 성능 검증
5. 실제 데이터로 테스트

In [1]:
#허깅페이스 패키지 설치
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 15.3MB/s 
     |████████████████████████████████| 890kB 37.1MB/s 
     |████████████████████████████████| 1.1MB 59.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=399bc6c9b4f3b117a1c580e5698ca748fcc4fb61cd255fe881ce1e05750b2db4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


##1. 네이버 영화 감성분석 데이터 로드

In [4]:
#네이버 영화 감성분석 데이터 로드
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [5]:
os.listdir('nsmc')

['code',
 'ratings_train.txt',
 'README.md',
 'ratings.txt',
 '.git',
 'synopses.json',
 'raw',
 'ratings_test.txt']

In [6]:
train = pd.read_table("nsmc/ratings_train.txt")
test = pd.read_table("nsmc/ratings_test.txt")
train[50:70]

,id,document,label
50,9063648,영화가 사람의 영혼을 어루만져 줄 수도 있군요 거친 세상사를 잠시 잊고 동화같은 영...,1
51,8272095,야 세르게이! 작은고추의 매운맛을 보여주마! 포퐁저그 콩진호가 간다,0
52,2345905,이렇게 가슴시리게 본 드라마가 또 있을까? 감동 그 자체!,1
53,7865630,"난또 저 꼬마애가 무슨 원한이 깊길래.,. 했더니 OO 그냥 혼자 나대다 OO걸 어...",0
54,7207064,재미있어요,1
55,5719655,전 좋아요,1
56,1651126,최고,0
57,7246040,너무 충격적이엇다. 기분을 완전히 푹 꺼지게 하는 느낌... 활력이라고는 하나도 없...,1
58,717775,심심한영화.,0
59,8317483,백봉기 언제나오나요?,1


##2.버트 모델에 맞는 인풋 만들기

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [8]:
# 문장을 버트 모델 인풋에 맞는 토큰값으로 바꿔줌
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[101, 9356, 11018, 31605, 31605, 110589, 71568, 118913, 11018, 9576, 119281, 9786, 79940, 23811, 40364, 9520, 23160, 102]


In [9]:
# 문장을 버트 모델 인풋에 맞는 토큰 형태로 표현해줌
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어', '##맞', '##는', '예', '##측', '카', '##리스', '##마', '없는', '악', '##역']


**네이버 영화 훈련 데이터 15만개를 버트에 맞는 인풋값으로 변형 해줄거임**

버트의 인풋은 토큰, 세그먼트, 마스크로 나눠짐

이 세 타입의 인풋이 버트 모형에 들어가면, 버트 모델에 맞게 임베딩 된다.

토큰 : 단어 사전의 위치값으로 표현

세그먼트 : 앞 문장인지, 뒷 문장인지 표현 (해당 예제에서는 인풋 문장이 하나만 들어가기 때문에 0으로 통일)

마스크: 문장이 유효한지 아닌지 (유효1 무효 0) 즉 패딩과 비패딩 부분을 구분

In [10]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['전', '##율', '##을', '일', '##으', '##키는', '영화', '.', '다시', '보고', '##싶', '##은', '영화']


In [11]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102]


In [12]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=128, pad_to_max_length=True))

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


위 예제를 정리하자면, 

1.문장을 버트용에 맞게 토크나이징 : "'전', '##율', '##을', '일', '##으', '##키는', '영화', '.', '다시', '보고', '##싶', '##은', '영화'

2.버트 인풋에 맞게 인코딩 : 101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102

3.버트 인풋 길이에 맞춰서 0으로 max padding

In [13]:
#세그먼트 인풋 / 인풋이 한 문장만 들어가기 때문에 0으로 통일
print([0]*128)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
#마스크 인풋
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (128 - valid_num) * [0]) # 비패딩과 패딩 부분을 구분

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


버트의 인풋은 토큰, 세그먼트, 마스크로 이루어져 있습니다.
"전율을 일으키는 영화. 다시 보고싶은 영화" 라는 문장을 가지고 예를 들면,

토큰 인풋 : [101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

세그먼트 인풋 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

마스크 인풋 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

이제 네이버 영화 평가 문장들을 전부 위와 같은 타입으로 바꿔보겠다

Huggingface에서의 순서는 [토큰인풋, 마스크인풋, 세그먼트인풋]

In [15]:
def convert_data(data_df):
  global tokenizer

  SEQ_LEN = 128 #버트에 들어갈 인풋의 길이
  tokens, masks, segments, targets = [], [], [], []

  for i in tqdm(range(len(data_df))):
    #문장을 토큰화 한다.
    token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')

    # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

    # 문장의 전후 관계를 구분해주는 세그먼트는 문장이 1개밖에 없으니까 모두 0 처리
    segment = [0]*SEQ_LEN

    # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

    # 정답(긍정:1, 부정:0)을 targets 변수에 저장해 줌
    targets.append(data_df[LABEL_COLUMN][i])

  #tokens, masks, segments, targets를 numpy array로 지정
  tokens = np.array(tokens)
  masks = np.array(masks)
  segments = np.array(segments)
  targets = np.array(targets)

  return [tokens, masks, segments], targets

In [16]:
#위에 정의한 convert_data 함수를 불러오는 함수 
def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
  data_x, data_y = convert_data(data_df)
  return data_x, data_y

SEQ_LEN = 128
BATCH_SIZE= 20
# 긍부정 문장 모두를 포함하고 있는 칼럼
DATA_COLUMN = "document"

# 긍정인지 부정인지 나타내는 레이블
LABEL_COLUMN = "label"

#train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)
#모델 성능을 검증할 test 데이터 또한 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 50000/50000 [00:16<00:00, 2989.59it/s]


##3.버트를 활용한 감성분석 모델 만들기

tpu 사용

In [17]:
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.107.242.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.107.242.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [18]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

# 아웃풋 shape은 [batch_size, 문장 길이, 768]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
bert_outputs = bert_outputs[1]
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1.0e-5), loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
sentiment_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 177853440   input_word_ids[0][0]             
                                                                 input_masks[0][0]     

감정 분석에 맞는 버트 모형을 리턴하는 함수 정의

TPU 활용 시, 함수로 묶어야 함

In [20]:
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025)

In [21]:
def create_sentiment_bert():
  
  #버트 프리트레인 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  
  #토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

  #인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
  return sentiment_model

In [22]:
bert_outputs = bert_outputs[1]
print(bert_outputs)

Tensor("strided_slice:0", shape=(768,), dtype=float32)


##4.훈련 및 성능 검증

In [23]:
if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  #함수를 strategy.scope로 묶어 줌
  with strategy.scope():
    sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=100, validation_data=(test_x, test_y))

else: #GPU 모드 훈련
  sentiment_model = create_sentiment_bert()
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=100, validation_data=(test_x, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Epoch 1/4
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/1500 [..............................] - ETA: 9:12:04 - loss: 0.6928 - accuracy: 0.4800 WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0098s vs `on_train_batch_end` time: 0.1515s). Check your callbacks.


1500/1500 [==============================] - ETA: 0s - loss: 0.4469 - accuracy: 0.7833WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0053s vs `on_test_batch_end` time: 0.0437s). Check your callbacks.


1500/1500 [==============================] - 315s 210ms/step - loss: 0.4469 - accuracy: 0.7833 - val_loss: 0.3581 - val_accuracy: 0.8397
Epoch 2/4
1500/1500 [==============================] - 261s 174ms/step - loss: 0.3366 - accuracy: 0.8501 - val_loss: 0.3252 - val_accuracy: 0.8569
Epoch 3/4
1500/1500 [==============================] - 261s 174ms/step - loss: 0.2941 - accuracy: 0.8719 - val_loss: 0.3069 - val_accuracy: 0.8660
Epoch 4/4
1500/1500 [==============================] - 260s 173ms/step - loss: 0.2595 - accuracy: 0.8894 - val_loss: 0.3053 - val_accuracy: 0.8703


In [24]:
#훈련 모델 예측 성능 체크(F1 socre)를 위한 기반 작업
def predict_convert_data(data_df):
  global tokenizer
  tokens, masks, segments = [], [], []

  for i in tqdm(range(len(data_df))):

    token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)
  
  tokens = np.array(tokens)
  masks = np.array(masks)
  segments = np.array(segments)
  return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_x = predict_convert_data(data_df)
  return data_x

test_set = predict_load_data(test)
test_set

100%|██████████| 50000/50000 [00:16<00:00, 3124.92it/s]


[array([[  101,  8911,   100, ...,     0,     0,     0],
        [  101,   144, 11490, ...,     0,     0,     0],
        [  101,  9303, 21711, ...,     0,     0,     0],
        ...,
        [  101,  8924, 67527, ...,     0,     0,     0],
        [  101,  9666, 14423, ...,     0,     0,     0],
        [  101,  9246, 32537, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [25]:
with strategy.scope():
  preds = sentiment_model.predict(test_set)

#부정이면 0, 긍정이면 1
preds

array([[0.97634333],
       [0.5835926 ],
       [0.7190106 ],
       ...,
       [0.47049677],
       [0.00377896],
       [0.21447322]], dtype=float32)

**F1 스코어 측정하기**

recall : 모델이 True라고 판정한 개수 / 전체 True 개수

precision : (모델이 True라고 판정한 개수) 중에 진짜 True 개수 / 모델이 True라고 판정한 개수

In [26]:
from sklearn.metrics import classification_report
y_true = test['label']

print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87     24827
           1       0.87      0.87      0.87     25173

    accuracy                           0.87     50000
   macro avg       0.87      0.87      0.87     50000
weighted avg       0.87      0.87      0.87     50000



In [27]:
import logging
tf.get_logger().setLevel(logging.ERROR)

##실제 데이터로 실습하기

In [40]:
def sentence_convert_data(data):
  global tokenizer
  tokens, masks, segments = [], [], []
  token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')

  num_zeros = token.count(0)
  mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
  segment = [0]*SEQ_LEN

  tokens.append(token)
  segments.append(segment)
  masks.append(mask)

  tokens = np.array(tokens)
  masks = np.array(masks)
  segments = np.array(segments)
  return [tokens, masks, segments]

def movie_evaluation_predict(sentence):
  data_x = sentence_convert_data(sentence)
  predict = sentiment_model.predict(data_x)
  predict_value = np.ravel(predict) #ravel -> 1차원으로 펼치는 함수
  predict_answer = np.round(predict_value,0).item()

  if predict_answer == 0:
    print("(부정 확률 : %.2f) 부정적인 영화 평가입니다." % (1-predict_value))
  elif predict_answer == 1:
    print("(긍정 확률 : %.2f) 긍정적인 영화 평가입니다." % predict_value)

In [41]:
movie_evaluation_predict("보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ")

(부정 확률 : 0.99) 부정적인 영화 평가입니다.


In [42]:
movie_evaluation_predict("오호 재밌다고 추천을 받아서 봤는데, 이거 완전 반전이네.")

(긍정 확률 : 0.60) 긍정적인 영화 평가입니다.


In [45]:
movie_evaluation_predict("캐비추 댓글이 많이 달려서 반신반의하며 봤는데 나름 괜찮았음")

(긍정 확률 : 0.98) 긍정적인 영화 평가입니다.


In [44]:
movie_evaluation_predict("스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다")

(긍정 확률 : 1.00) 긍정적인 영화 평가입니다.
